# auto-sklearn

ディープラーニングモデルを自動構築するPythonライブラリ

## データセットの読み込み(ボストン住宅価格)

In [1]:
from sklearn.datasets import load_boston
import pandas as pd
from sklearn.model_selection import train_test_split

boston = load_boston()
df_x_boston = pd.DataFrame(boston['data'], columns=boston['feature_names'])
df_y_boston = pd.DataFrame(boston['target'], columns=['MEDV'])
df_boston = pd.concat([df_x_boston, df_y_boston], axis=1)

x_train, x_test, y_train, y_test = train_test_split(df_x_boston[['RM']], df_y_boston, test_size=0.3, random_state=0)

df_train = pd.concat([x_train, y_train], axis=1)
df_test = pd.concat([x_test, y_test], axis=1)

In [2]:
x_train

,RM
141,5.019
272,6.538
135,6.335
298,6.345
122,5.961
...,...
323,5.708
192,7.178
117,6.021
47,6.030


In [3]:
y_train

,MEDV
141,14.4
272,24.4
135,18.1
298,22.5
122,20.5
...,...
323,18.5
192,36.4
117,19.2
47,16.6


In [4]:
from pprint import pprint
import sklearn.metrics

import autosklearn.regression

In [5]:
tmp_folder='/tmp/sample/autosklearn'
!rm -rf {tmp_folder}

In [6]:
automl = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=1000,
    per_run_time_limit=100,
    tmp_folder=tmp_folder,
    memory_limit=None,
)
automl.fit(x_train, y_train, dataset_name='boston')

[WARNING] [2022-06-10 12:04:12,146:Client-AutoMLSMBO(1)::boston] Dataset boston already in meta-data. Removing occurence.


/usr/local/lib/python3.8/dist-packages/autosklearn/metalearning/metalearning/meta_base.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.metafeatures = self.metafeatures.append(metafeatures)
/usr/local/lib/python3.8/dist-packages/autosklearn/metalearning/metalearning/meta_base.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.algorithm_runs[metric].append(runs)


AutoSklearnRegressor(memory_limit=None, per_run_time_limit=100,
                     time_left_for_this_task=1000,
                     tmp_folder='/tmp/sample/autosklearn')

In [7]:
print(automl.leaderboard())

          rank  ensemble_weight           type      cost  duration
model_id                                                          
309          1             0.32     libsvm_svr  0.342060  0.564120
331          2             0.06     libsvm_svr  0.347260  0.493533
129          3             0.24            mlp  0.350058  0.502457
294          4             0.02     libsvm_svr  0.352592  0.565476
228          5             0.10     libsvm_svr  0.367268  0.542190
14           6             0.26  random_forest  0.369234  1.143785


In [8]:
train_pred = automl.predict(x_train)
test_pred = automl.predict(x_test)

print("Train R2 score:", sklearn.metrics.r2_score(y_train, train_pred))
print("Train R2 score:", sklearn.metrics.r2_score(y_test, test_pred))

Train R2 score: 0.6180940605891598
Train R2 score: 0.5056102761133405
